Import a network trained on fish detection

In [ ]:
! git clone https://github.com/kwea123/fish_detection.git

Cloning into 'fish_detection'...
remote: Enumerating objects: 254, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 254 (delta 0), reused 1 (delta 0), pack-reused 250
Receiving objects: 100% (254/254), 231.35 MiB | 16.38 MiB/s, done.
Resolving deltas: 100% (132/132), done.
Checking out files: 100% (46/46), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Detecting morts

Try to do something similar to this: https://pyimagesearch.com/2019/03/11/liveness-detection-with-opencv/

- Save the predictions of the detector as new images in a new folder to create a training dataset
- Manually separate images of live fish from images of morts
- Train a network to distinguish between the two





## Step 1
Perform detection on the video and save predicted images
### NOT RUN

In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image
import time
import cv2
import matplotlib.pyplot as plt

class Object_Detector():
    def __init__(self, model_path):
        self.__load_model(model_path)
        print('model loaded')

    def __load_model(self, model_path):
        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.compat.v1.GraphDef()
            with tf.compat.v2.io.gfile.GFile(model_path, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')

        config = tf.compat.v1.ConfigProto()
        config.gpu_options.allow_growth= True

        with self.detection_graph.as_default():
            self.sess = tf.compat.v1.Session(config=config, graph=self.detection_graph)
            self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
            self.detection_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
            self.detection_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
            self.detection_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
            self.num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')

        # load label_dict
        self.label_dict = {1: 'fish'}
        # warmup
        self.detect_image(np.ones((600, 600, 3)))


    def detect_image(self, image_np, output_path = None, score_thr=0.5, saved=0, print_time=False, fish_boxes=None, box_thr=0.7, cur_frame=0):
        image_w, image_h = image_np.shape[1], image_np.shape[0]
    
        # Actual detection.
        (boxes, scores, classes, num) = self.sess.run(
          [self.detection_boxes, self.detection_scores, self.detection_classes, self.num_detections],
          feed_dict={self.image_tensor: np.expand_dims(image_np, axis=0)})
        # Visualization of the results of a detection.
        for i, box in enumerate(boxes[scores>score_thr]):
            
            top_left = (int(box[1]*image_w), int(box[0]*image_h))
            bottom_right = (int(box[3]*image_w), int(box[2]*image_h))
            fish = image_np[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
            fish = cv2.cvtColor(fish, cv2.COLOR_RGB2BGR)

            # write the frame to disk
            p = os.path.sep.join([output_path,
              "{}.png".format(saved)])
            cv2.imwrite(p, fish)
            #print("[INFO] saved {} to disk".format(p))

            cv2.rectangle(image_np, top_left, bottom_right, (0,255,0), 3)
            cv2.putText(image_np, self.label_dict[int(classes[0,i])], top_left, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

        return image_np

In [ ]:
%cd fish_detection
# MODEL_PATH = 'fish_ssd_fpn_graph/frozen_inference_graph.pb'
MODEL_PATH = 'fish_inception_v2_graph/frozen_inference_graph.pb'
object_detector = Object_Detector(MODEL_PATH)

[Errno 2] No such file or directory: 'fish_detection'
/content/fish_detection
model loaded


In [ ]:
!unzip /content/drive/MyDrive/PoliTOcean/fish_videos_train.zip 

Archive:  /content/drive/MyDrive/PoliTOcean/fish_videos_train.zip
   creating: fish_videos_train/
  inflating: fish_videos_train/fish_practice_video_1 (1080p).mp4  
  inflating: fish_videos_train/fish_practice_video_2 (1080p).mp4  
  inflating: fish_videos_train/fish_practice_video_3 (1080p).mp4  
  inflating: fish_videos_train/fish_practice_video_4 (1080p).mp4  
  inflating: fish_videos_train/fish_practice_video_6 (1080p).mp4  
  inflating: fish_videos_train/fish_practice_video_7 (1080p).mp4  
  inflating: fish_videos_train/fish_practice_video_8 (1080p).mp4  
  inflating: fish_videos_train/fish_practice_video_9 (1080p).mp4  


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import os
from tqdm import tqdm

input_dir = "/content/fish_detection/fish_videos_train"

directory = os.fsencode(input_dir)
    
for file in tqdm(os.listdir(directory)):
    input_path = os.fsdecode(file)

    print("### Working on file " + input_path)

    output_path = "/content/drive/MyDrive/PoliTOcean/out/" + input_path.split('.')[0]
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # Create a VideoCapture object and read from input file
    # If the input is the camera, pass 0 instead of the video file name
    cap = cv2.VideoCapture(input_dir + '/' + input_path)
    # Check if camera opened successfully
    if (cap.isOpened()== False): 
      print("Error opening video stream or file")
    # Default resolutions of the frame are obtained.The default resolutions are system dependent.
    # We convert the resolutions from float to integer.
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    # Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
    out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
    saved=0
    # Read until video is completed
    while(cap.isOpened()):
      # Capture frame-by-frame
      ret, frame = cap.read()
      if ret == True:
        if saved % 1 == 0:
          img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
          img_ = object_detector.detect_image(img, score_thr=0.5, output_path=output_path, saved=saved/1)    
          # Write the frame into the file 'output.avi'
          out.write(img_)
        saved += 1

      # Break the loop
      else: 
        break

# When everything done, release the video capture and video write objects
cap.release()
out.release()

# Closes all the frames
cv2.destroyAllWindows()


  0%|          | 0/8 [00:00<?, ?it/s]

### Working on file fish_practice_video_2 (1080p).mp4


 12%|█▎        | 1/8 [02:47<19:34, 167.82s/it]

### Working on file fish_practice_video_8 (1080p).mp4


 25%|██▌       | 2/8 [13:25<44:26, 444.42s/it]

...

 75%|███████▌  | 6/8 [51:31<22:17, 668.61s/it]

### Working on file fish_practice_video_3 (1080p).mp4


 88%|████████▊ | 7/8 [54:37<08:30, 510.73s/it]

### Working on file fish_practice_video_4 (1080p).mp4


100%|██████████| 8/8 [1:02:11<00:00, 466.44s/it]


In [ ]:
! zip -r /content/drive/MyDrive/PoliTOcean/out.zip /content/drive/MyDrive/PoliTOcean/out

## Step 2
Manually divide saved images into predictions of alive and mort fish (not a funny part)

## Step 3
Create a simple model to be trained on the saved data

#### NOT RUN

In [ ]:
!unzip /content/drive/MyDrive/PoliTOcean/alive.zip -d ./alive
!unzip /content/drive/MyDrive/PoliTOcean/morts.zip -d ./morts
!unzip /content/drive/MyDrive/PoliTOcean/fish_videos_test.zip -d ./fish_videos_test

In [ ]:
!ls ./alive/alive | wc -l
!ls ./morts/morts | wc -l

In [ ]:
###
# Model definition
###

# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K
class LivenessNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1
		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

    # first CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(16, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(16, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
		# second CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(32, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(32, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
  	# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(64))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))
		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))
		# return the constructed network architecture
		return model

In [ ]:
###
# Training the model
###

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os

dataset_alive = "alive/alive"
dataset_morts = "morts/morts"
model_path = "model"
le_path = "label_encoder"
plot = "loss_plot"

# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-4
BS = 8
EPOCHS = 100

print("[INFO] loading images...")
data = []
labels = []

for label, dataset in enumerate([dataset_alive, dataset_morts]):
  imagePaths = list(paths.list_images(dataset))
  for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (32, 32))
    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)

# convert the data into a NumPy array, then preprocess it by scaling
# all pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
# encode the labels (which are currently strings) as integers and then
# one-hot encode them
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels, 2)

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.25, random_state=42)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")
# initialize the optimizer and model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model = LivenessNet.build(width=32, height=32, depth=3,
	classes=len(le.classes_))
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the network
print("[INFO] training network for {} epochs...".format(EPOCHS))
H = model.fit(x=aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS)

# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x=testX, batch_size=BS)
# save the network to disk
model.save(model_path, save_format="h5")
# save the label encoder to disk
f = open(le_path, "wb")
f.write(pickle.dumps(le))
f.close()
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, EPOCHS), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, EPOCHS), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, EPOCHS), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, EPOCHS), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(plot)

[INFO] loading images...
[INFO] compiling model...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[INFO] training network for 100 epochs...
Epoch 1/100
300/300 [==============================] - 17s 15ms/step - loss: 0.7702 - accuracy: 0.6708 - val_loss: 0.6946 - val_accuracy: 0.5169
Epoch 2/100
300/300 [==============================] - 4s 13ms/step - loss: 0.5771 - accuracy: 0.7865 - val_loss: 0.3873 - val_accuracy: 0.8826
Epoch 3/100
300/300 [==============================] - 4s 13ms/step - loss: 0.5098 - accuracy: 0.8191 - val_loss: 0.3134 - val_accuracy: 0.9014
Epoch 4/100
300/300 [==============================] - 4s 13ms/step - loss: 0.4653 - accuracy: 0.8317 - val_loss: 0.3334 - val_accuracy: 0.8851
Epoch 5/100
300/300 [==============================] - 4s 13ms/step - loss: 0.4136 - accuracy: 0.8463 - val_loss: 0.3018 - val_accuracy: 0.8864
Epoch 6/100
300/300 [==============================] - 4s 13ms/step - loss: 0.4067 - accuracy: 0.8571 - val_loss: 0.3029 - val_accuracy: 0.9089
Epoch 7/100
300/300 [==============================] - 4s 13ms/step - loss: 0.4049 - accuracy

## Deploy trained model on top of the detection network

To run to perform detection

In [ ]:
%cd fish_detection

/content/fish_detection


In [ ]:
!unzip /content/drive/MyDrive/PoliTOcean/fish_videos_test.zip

Archive:  /content/drive/MyDrive/PoliTOcean/fish_videos_test.zip
   creating: fish_videos_test/
  inflating: fish_videos_test/fish_practice_video_10 (1080p).mp4  
  inflating: fish_videos_test/fish_practice_video_11 (1080p).mp4  
  inflating: fish_videos_test/fish_practice_video_12 (1080p).mp4  
  inflating: fish_videos_test/fish_practice_video_5 (1080p).mp4  


In [ ]:
###
# Define object detector class and methods
###

import numpy as np
import tensorflow as tf
from PIL import Image
import time
import cv2
import matplotlib.pyplot as plt
from imutils.video import VideoStream
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import argparse
import imutils
import pickle
import time
import os


class Object_Detector():
    def __init__(self, model_path):
        self.__load_model(model_path)
        print('model loaded')

    def __load_model(self, model_path):
        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.compat.v1.GraphDef()
            with tf.compat.v2.io.gfile.GFile(model_path, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')

        config = tf.compat.v1.ConfigProto()
        config.gpu_options.allow_growth= True

        with self.detection_graph.as_default():
            self.sess = tf.compat.v1.Session(config=config, graph=self.detection_graph)
            self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
            self.detection_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
            self.detection_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
            self.detection_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
            self.num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')

        # load label_dict
        self.label_dict = {1: 'fish'}
        
        # warmup
        self.detect_image(np.ones((600, 600, 3)))


    def detect_image(self, image_np, score_thr=0.5, aliveness_model=None, le=None):
        image_w, image_h = image_np.shape[1], image_np.shape[0]

        # Actual detection.
        t = time.time()
        (boxes, scores, classes, num) = self.sess.run(
          [self.detection_boxes, self.detection_scores, self.detection_classes, self.num_detections],
          feed_dict={self.image_tensor: np.expand_dims(image_np, axis=0)})

        
        # Visualization of the results of a detection.
        for i, box in enumerate(boxes[scores>score_thr]):
            top_left = (int(box[1]*image_w), int(box[0]*image_h))
            bottom_right = (int(box[3]*image_w), int(box[2]*image_h))
            fish = image_np[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
            fish = cv2.cvtColor(fish, cv2.COLOR_RGB2BGR)
            fish = cv2.resize(fish, (32, 32))
            fish = fish.astype("float") / 255.0
            fish = img_to_array(fish)
            fish = np.expand_dims(fish, axis=0)
            # pass the face ROI through the trained liveness detector
            # model to determine if the face is "real" or "fake"
            preds = aliveness_model.predict(fish)[0]
            #j = np.argmax(preds)
            if preds[1] > 0.01:
              j = 1
            else:
              j = 0 
            label = le.classes_[j]

            # if label == 0:
            #   cv2.rectangle(image_np, top_left, bottom_right, (0,255,0), 3)
            #   cv2.putText(image_np, "alive: " + str(preds), top_left, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

            # else:
            if label != 0:
              cv2.rectangle(image_np, top_left, bottom_right, (255,0,0), 3)
              cv2.putText(image_np, "mort " + str(preds), top_left, cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
        img_float32 = np.uint8(image_np)
        image_np = cv2.cvtColor(img_float32, cv2.COLOR_RGB2BGR)
        return image_np

In [ ]:
!mv /content/label_encoder /content/drive/MyDrive/PoliTOcean/label_encoder

mv: cannot stat '/content/label_encoder': No such file or directory


In [ ]:
###
# Load pretrained models
###

model_path = "/content/drive/MyDrive/PoliTOcean/model"
le_path = "/content/drive/MyDrive/PoliTOcean/label_encoder"

# model_path = "/content/model"
# le_path = "/content/label_encoder"

aliveness_model = load_model(model_path)
le = pickle.loads(open(le_path, "rb").read())
MODEL_PATH = 'fish_inception_v2_graph/frozen_inference_graph.pb'
object_detector = Object_Detector(MODEL_PATH)

model loaded


### TODO


In [ ]:
video_input_link = '/content/fish_detection/fish_videos_test/Task 2.2 EXPLORER Station 1 Fish Video.mp4'
video_output_link = '/content/drive/MyDrive/PoliTOcean/out/outpy.avi'

In [ ]:
###
# Load video and perform detections
###

import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture(video_input_link)

# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

# Default resolutions of the frame are obtained.The default resolutions are system dependent.
# We convert the resolutions from float to integer.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
out = cv2.VideoWriter(video_output_link,cv2.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))

# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_ = object_detector.detect_image(img, score_thr=0.7, aliveness_model=aliveness_model, le=le)
    
    # Write the frame into the file 'output.avi'
    out.write(img_)

  # Break the loop
  else: 
    break


# When everything done, release the video capture and video write objects
cap.release()
out.release()

# Closes all the frames
cv2.destroyAllWindows()

In [ ]:
# Visualize saved video
from moviepy.editor import *

path="/content/drive/MyDrive/PoliTOcean/out/outpy.avi" 

clip=VideoFileClip(path)
clip.ipython_display(width=1000, max_duration=100000000000000)

100%|█████████▉| 1800/1801 [00:51<00:00, 34.80it/s]
